# SPSectors Rolling, M = 120

In [5]:
import pandas as pd
import numpy as np

In [6]:
# import data

SPSectorsPandas = pd.read_csv("data/SPSectors.txt", delim_whitespace = True)

In [7]:
SPSectorsPandas.head()

,%date,T-bill,Energy,Material,Industrials,Cons-Discr.,Cons-Staples,Helth-Care,Financials,Inf-Tech.,Telecom,Utilities,S\&P500
0,19810130,0.0100,-0.1227,-0.0320,-0.0308,0.0038,0.0127,-0.0238,-0.0243,-0.1160,-0.0452,-0.0289,-0.0536
1,19810227,0.0107,0.0024,0.0030,0.0526,0.0446,0.0332,0.0015,0.0135,0.0110,-0.0222,-0.0359,0.0107
2,19810331,0.0132,-0.0338,0.1203,0.0577,0.0718,0.0314,0.0192,0.0740,0.0422,0.0677,0.0197,0.0253
3,19810430,0.0108,-0.0643,-0.0333,-0.0321,-0.0095,0.0104,-0.0015,-0.0028,-0.0054,0.0022,-0.0377,-0.0311
4,19810529,0.0106,-0.0570,-0.0047,-0.0078,0.0310,-0.0063,0.0017,0.0297,-0.0197,0.0674,0.0065,-0.0040


In [8]:
SPSectors = SPSectorsPandas.to_numpy()[:, 1:]

In [9]:
rows, cols = SPSectors.shape
n = cols - 1

In [10]:
# set parameters

# Gamma : How risk averse an investor is, gamma >= 0
gammaValue = [1, 2, 3, 4, 5, 10]

# estimation window; how long we will estimate for
window = 120

In [11]:
# portfolio policies

pf = {
    "ew" : np.empty((n, rows - window)),
    "mv" : np.empty((n, rows - window))
}

# portfolio weights before rebalancing
pfBuyHold = {
    "ew" : np.empty((n, rows - window)),
    "mv" : np.empty((n, rows - window))
}

# out of sample returns
outSample = {
    "ew" : np.empty((1, rows - window)),
    "mv" : np.empty((1, rows - window))
}

In [12]:
# seperate risky from risk-free asset

riskFreeReturns = SPSectors[:, 0] # risk-free asset column
riskyReturns = SPSectors[:, 1:cols] # risky asset column, includes risk factor
# riskFactor = SPSectors[:, end - RISK_FACTOR + 1: cols] # risk factor column

In [13]:
M = 120
T = len(riskyReturns)

nSubsets = 1 if M == T else T - M

In [14]:
# 1/N equal weight model

# @param n : number of columns
# @returns : np.array of size [n - 1, 1]
def ew(n):
    return 1/n * np.ones((n - 1, 1))

In [22]:
# mean-variance model

def mv(AMLE, invSigmaMLE, n):
    return 1/AMLE * invSigmaMLE.dot(np.ones((n - 1, 1)))

In [16]:
# w : array of weights
# j : current shift position
def buyHold(w, j):

    a = (1 - sum(w)) * (1 + riskFreeReturns[window + j])
    b = (1 + (riskyReturns[window + j, :].T + riskFreeReturns[window + j]))[np.newaxis].T
    trp =a + w[np.newaxis].dot(b)
    
    return ((w * (1 + (riskyReturns[window + j, :]).T + riskFreeReturns[window + j])) / trp)


In [17]:
# w : array of weights
# j : current shift position
def outOfSampleReturns(w, j):
    return w.T.dot(riskyReturns[window + j, :][np.newaxis].T)

In [18]:
# x : array of out of sample data
def sharpeRato(x):
    mean = np.mean(x.T)
    std = np.std(x.T, ddof=1)
    
    if (abs(mean) > pow(10, -16)):
        sr = mean / std;
    else:
        sr = None
            
    return sr

In [23]:
# rolling sample

for shift in range(0, nSubsets):

    riskySubset = riskyReturns[shift:window + shift, :]
    riskFreeSubset = riskFreeReturns[shift:window + shift]
    subset = np.column_stack((riskFreeSubset, riskySubset))
    
    mu = np.array([np.mean(riskFreeSubset)])
    mu = np.append(mu, np.vstack(riskySubset.mean(axis = 0)))
    
    totalSigma = np.cov(subset.T)
    sigma = (M - 1) / (M - n - 1 - 2) * np.cov(riskySubset.T)
    
    sigmaMLE = (M - 1) / M * np.cov(riskySubset.T)
    invSigmaMLE = np.linalg.inv(sigmaMLE)
    AMLE = np.ones((1, cols - 1)).dot(invSigmaMLE).dot(np.ones((cols - 1, 1)))
    
    # 1/N
    alphaTew = ew(cols)
    pf["ew"][:, shift] = alphaTew[:, 0]
    
    # mean-variance
    alphaMV = mv(AMLE, invSigmaMLE, cols)
    pf["mv"][:, shift] = alphaMV[:, 0]

#     buy and hold
    if shift == 0:
        pfBuyHold["ew"][:, shift]= alphaTew[:, 0]
        pfBuyHold["mv"][:, shift]= alphaMV[:, 0]
    else:
        pfBuyHold["ew"][:, shift] = buyHold(pf["ew"][:, shift - 1], shift)
        pfBuyHold["mv"][:, shift] = buyHold(pf["mv"][:, shift - 1], shift)
        
    if (nSubsets > 1):
#   out of sample returns
        outSample["ew"][:, shift] = outOfSampleReturns(alphaTew, shift)[:, 0]
        outSample["mv"][:, shift] = outOfSampleReturns(alphaMV, shift)[:, 0]

In [24]:
# test 1/N naive portfolio policy

sr = sharpeRato(outSample["ew"])
mv = sharpeRato(outSample["mv"])

print(sr)
print(mv)

0.18761963181287292
0.0819981240886543
